# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output2'
TRAIT = 'Thymoma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

utils.py has been loaded


In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [3]:
trait_subdir = "TCGA_Thymoma_(THYM)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THYM.sampleMap_THYM_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THYM.sampleMap_HiSeqV2_PANCAN.gz')

In [4]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [5]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [6]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode',
 'bcr_patient_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [7]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'form_completion_date', 'gender', 'height', 'histolo

In [10]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']


Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [11]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [47.0, 66.0, 45.0, 51.0, 76.0],
 'days_to_birth': [-17528.0, -24460.0, -16728.0, -18733.0, -27827.0],
 'days_to_collection': [1779.0, 1681.0, 4923.0, 611.0, 2571.0],
 'days_to_death': [nan, nan, 2488.0, 775.0, 2910.0],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [2086.0, 1799.0, nan, 540.0, 2756.0],
 'year_of_initial_pathologic_diagnosis': [2009, 2009, 2000, 2012, 2006]}

In [12]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [13]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [14]:
gender_col = 'gender'

In [15]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [16]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

13 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('RPL31P11', 2), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
10 input query terms found dup hits:	[('SUGT1P1', 2), ('PTPRVP', 2), ('SNORA62', 3), ('IFITM4P', 7), ('HLA-DRB6', 2), ('FUNDC2P2', 2), ('
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
11 input query terms found dup hits:	[('PIP5K1P1', 2), ('HBD', 2), ('PPP1R2P1', 9), ('HSD17B7P2', 2), ('RPSAP9', 2), ('SNORD68', 2), ('SN
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
15 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('ESPNP', 2), ('HBBP1
158 input query terms found no hit:	[

In [17]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thymoma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3G-AB0O-01,1,47.0,1,-3.523092,-1.918126,-0.531035,0.172928,-0.989278,-0.98941,0.543606,...,-0.511086,-0.086682,-0.748878,-0.269917,1.363067,1.17161,-0.992077,-1.857627,-0.052595,0.500067
TCGA-3G-AB0Q-01,1,66.0,1,-1.229792,2.802774,-0.531035,0.302428,-0.153078,0.11219,0.434706,...,1.363214,-0.086682,0.011122,3.437683,1.371367,3.39781,-0.058577,0.776973,0.146905,-0.547633
TCGA-3G-AB0T-01,1,45.0,1,-2.102092,-3.145926,-0.531035,0.121728,-1.219978,-1.25601,0.461306,...,-1.448986,0.535118,-0.748878,-1.587117,2.297467,-0.65959,-1.021877,0.077173,1.154605,0.809667
TCGA-3G-AB14-01,1,51.0,1,-0.743092,3.065074,0.972665,-0.045572,-0.621278,-0.47431,0.684306,...,-0.587086,-0.086682,0.190622,-1.587117,1.469667,2.46181,0.228423,-1.282927,0.882805,-0.762233
TCGA-3G-AB19-01,1,76.0,0,-1.432492,-1.164326,-0.177835,-0.126172,0.227822,0.30389,0.044306,...,-2.090786,-0.086682,0.506322,0.099583,0.993767,3.25411,0.058723,0.828273,0.279705,-1.407333


In [18]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 121 samples.


In [19]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thymoma', the least common label is '0' with 2 occurrences. This represents 1.65% of the dataset.
The distribution of the feature 'Thymoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 59.0
  75%: 68.0
Min: 17.0
Max: 84.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 58 occurrences. This represents 47.93% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [20]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [21]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Haoyang/Thymoma/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Thymoma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE131027', 'GSE23540', 'GSE29695', 'GSE42977', 'GSE55852']

Repeat the below steps for all the accession numbers

In [83]:
cohort = accession_num = "GSE55852"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Thymoma\\GSE55852\\GSE55852_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Thymoma\\GSE55852\\GSE55852_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [84]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"GTF2I Mutations are Common in Thymic Epithelial Tumors"
!Series_summary	"Within a project aim to define the genomic aberration of thymic epithelial tumors, we performed array CGH in 65 thymic epithelial tumors. Tumor samples were collected during surgery or by image-guided biopsies and immediately frozen. Section from frozen material were cut and stained with Haematoxylin and Eosin. A pathologist reviewed the slides and selected only cases with >80% of cancer cells."
!Series_overall_design	"Copy number aberrations of a series of 65 thymic epithelial tumors were evaluated using array CGH. Differences in copy number aberrations between different histotypes were evaluated. Significant regions of CN aberrations were defined using GISTIC algorithms."


In [85]:
clinical_data.head()

,!Sample_geo_accession,GSM1347258,GSM1347259,GSM1347260,GSM1347261,GSM1347262,GSM1347263,GSM1347264,GSM1347265,GSM1347266,...,GSM1347313,GSM1347314,GSM1347315,GSM1347316,GSM1347317,GSM1347318,GSM1347319,GSM1347320,GSM1347321,GSM1347322
0,!Sample_characteristics_ch1,tumor: Thymic Carcinoma,tumor: Thymic Carcinoma,tumor: Thymic Carcinoma,tumor: Thymic Carcinoma,tumor: Thymoma,tumor: Thymic Carcinoma,tumor: Thymoma,tumor: Thymic Carcinoma,tumor: Thymoma,...,tumor: Thymic Carcinoma,tumor: Thymoma,tumor: Thymic Carcinoma,tumor: Thymic Carcinoma,tumor: Thymoma,tumor: Thymic Carcinoma,tumor: Thymoma,tumor: Thymoma,tumor: Thymic Carcinoma,tumor: Thymic Carcinoma
1,!Sample_characteristics_ch1,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M,Sex: F,Sex: F,Sex: M,...,Sex: F,Sex: F,Sex: M,Sex: F,Sex: F,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M
2,!Sample_characteristics_ch1,who: TC,who: TC,who: TC,who: NEC,who: B3,who: TC,who: B3,who: TC,who: B2,...,who: TC,who: A,who: NEC,who: TC,who: B2,who: TC,who: TC,who: B3,who: TC,who: TC
3,!Sample_characteristics_ch1,Stage: IVB,Stage: IVB,Stage: IVB,Stage: IVB,Stage: IVB,Stage: IVB,gtf2i: WT,Stage: IVB,Stage: IVB,...,Stage: IVB,Stage: II,NaN,NaN,NaN,NaN,NaN,"diploid regions: chr2, chr8, chr10, chr17",NaN,"diploid regions: chr2, chr4, chr5, chr8, chr12..."
4,!Sample_characteristics_ch1,gtf2i: WT,gtf2i: WT,gtf2i: WT,gtf2i: WT,gtf2i: WT,gtf2i: WT,NaN,gtf2i: WT,NaN,...,gtf2i: WT,gtf2i: WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tumor: Thymic Carcinoma', 'tumor: Thymoma'],
 1: ['Sex: M', 'Sex: F'],
 2: ['who: TC', 'who: NEC', 'who: B3', 'who: B2', 'who: AB', 'who: A'],
 3: ['Stage: IVB',
  'gtf2i: WT',
  'Stage: IIA',
  'Stage: III',
  'gtf2i: MUT',
  'Stage: IIB',
  'Stage: IVA',
  'Stage: I',
  'Stage: II',
  nan,
  'diploid regions: chr2, chr8, chr10, chr17',
  'diploid regions: chr2, chr4, chr5, chr8, chr12, chr13, chr14, chr15'],
 4: ['gtf2i: WT',
  nan,
  'diploid regions: chr2, chr3, chr4, chr6, chr7, chr9, chr10, chr11, chr13, chr14',
  'gtf2i: MUT',
  'gtf2i: WY',
  'diploid regions: chr2, chr3, chr4, chr10, chr11, chr12, chr13, chr15, chr16, chr17',
  'diploid regions: chr2, chr3, chr4, chr11, chr17'],
 5: [nan,
  'diploid regions: chr1, chr2, chr3, chr4, chr5, chr10, chr11, chr13, chr14, chr16, chr17',
  'diploid regions: chr2, chr3, chr4, chr5, chr13, chr14, chr15',
  'diploid regions: chr4, chr5, chr14, chr15',
  'diploid regions: chr2, chr4, chr8, chr10, chr11, chr13, chr17, chr18, chr19, c

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [87]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Thymoma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Thymoma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. T

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [88]:
is_gene_availabe = True
trait_row = 0
age_row = None
gender_row = 1

trait_type = 'binary'

In [89]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [93]:
def convert_trait(value):
    if value in [ 'tumor: Thymoma']:
        return 1  # Presence of thymoma
    else:
        return 0  # Absence or unknown

# def convert_trait(value):
#     # Since 'Thymoma' types are categorical, we'll keep it as is.
#     return value
def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format
def convert_gender(value):
    if value == 'Sex: M':
        return 'Male'
    elif value == 'Sex: F':
        return 'Female'
    else:
        return None  # Handle unknown or invalid values

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [94]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1347258,GSM1347259,GSM1347260,GSM1347261,GSM1347262,GSM1347263,GSM1347264,GSM1347265,GSM1347266,GSM1347267,...,GSM1347313,GSM1347314,GSM1347315,GSM1347316,GSM1347317,GSM1347318,GSM1347319,GSM1347320,GSM1347321,GSM1347322
Thymoma,0,0,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,1,1,0,0
Gender,Male,Male,Female,Male,Male,Male,Female,Female,Male,Female,...,Female,Female,Male,Female,Female,Male,Male,Female,Male,Male


### Genetic data preprocessing and final filtering

In [95]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1347258,GSM1347259,GSM1347260,GSM1347261,GSM1347262,GSM1347263,GSM1347264,GSM1347265,GSM1347266,GSM1347267,...,GSM1347313,GSM1347314,GSM1347315,GSM1347316,GSM1347317,GSM1347318,GSM1347319,GSM1347320,GSM1347321,GSM1347322
ID,,,,,,,,,,,,,,,,,,,,,
1,0.0113,0.0688,0.0428,0.11,0.00467,0.0211,0.0335,-0.028200,0.0393,0.0784,...,0.0964,0.165,-0.00105,0.0615,0.0643,0.0397,0.2,0.0769,-0.053,0.0508
2,0.1660,0.0000,0.0000,0.00,0.00000,0.1990,0.0000,-0.057400,-0.0806,0.0000,...,0.4140,0.360,0.00000,0.2770,-0.1330,-0.4690,0.0,-0.0823,0.000,-0.1940
3,0.3040,0.0000,0.0000,0.00,0.00000,0.0489,0.0000,0.000236,0.0000,0.0000,...,-0.1270,0.745,0.00000,-0.1390,-0.1630,-0.3310,0.0,0.0000,-0.115,0.0000
4,0.0533,0.0000,0.0000,0.00,0.00000,0.0000,0.0000,0.074500,0.0000,0.0000,...,0.2390,0.945,0.00000,-0.1070,-0.0744,-0.0995,0.0,-0.2550,0.000,0.0000
5,0.0000,0.0000,0.0000,0.00,0.00000,0.0000,0.0000,0.435000,0.0000,0.0000,...,0.2450,0.693,0.00000,0.0000,-0.1830,-0.3410,0.0,-0.0884,-0.075,-0.3650


In [96]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [97]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']\n"

If not required, jump directly to the gene normalization step

In [98]:
requires_gene_mapping = True

In [99]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '4', '5'], 'COL': ['532', '532', '532', '532', '532'], 'ROW': [340.0, 338.0, 336.0, 334.0, 332.0], 'NAME': [nan, nan, nan, nan, nan], 'SPOT_ID': ['HsCGHBrightCorner', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'RANGE_GB': [nan, nan, nan, nan, nan], 'RANGE_START': [nan, nan, nan, nan, nan], 'RANGE_END': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [100]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'COL': ['532', '532', '532', '532', '532'], 'ROW': [340.0, 338.0, 336.0, 334.0, 332.0], 'NAME': [nan, nan, nan, nan, nan], 'SPOT_ID': ['HsCGHBrightCorner', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'RANGE_GB': [nan, nan, nan, nan, nan], 'RANGE_START': [nan, nan, n

In [101]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [102]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [103]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [104]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 65 samples.


In [105]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thymoma', the least common label is '0' with 22 occurrences. This represents 33.85% of the dataset.
The distribution of the feature 'Thymoma' in this dataset is fine.

For the feature 'Gender', the least common label is 'Male' with 29 occurrences. This represents 44.62% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [106]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [107]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE123088'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE123088,True,True,False,True,True,192,
1,GSE123086,True,True,False,True,True,154,


In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)